In [5]:
from binance.client import Client
import pandas as pd
import plotly.graph_objects as go
from pon import api,secret
import numpy as np
import schedule
import time
from plotly.subplots import make_subplots
import pandas_ta as ta
import warnings
warnings.filterwarnings("ignore")


In [6]:
def get_price_data_binance(ticker:str,limit:int,interval = Client.KLINE_INTERVAL_1DAY)->pd.DataFrame:
    client = Client(api, secret)
    df = pd.DataFrame(client.get_klines(symbol=ticker, interval=interval, limit=limit))
    df.columns=['date','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    return df
    
def get_list_of_up_imbalances(df:pd.DataFrame)->list:
    up_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['high'] < next_candle['low']:
            up_imbalances.append([round((prev_candle['high']+next_candle['low'])/2,2), prev_candle['high'],df.iloc[i-1]['date']])
    return up_imbalances

def get_list_of_down_imbalances(df:pd.DataFrame)->list:
    down_imbalances = []
    for i in range(2, len(df)):
        prev_candle = df.iloc[i-2]
        next_candle = df.iloc[i]
        if prev_candle['low'] > next_candle['high']:
            down_imbalances.append([round((prev_candle['low']+next_candle['high'])/2,2) , prev_candle['low'] , df.iloc[i-1]['date']])
    return down_imbalances


def delete_up_imbalance(up_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in up_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2]  and (candle['close'] < imbalance[0] or candle['low'] < imbalance[0]):
                up_imbalances.remove(imbalance)
    return up_imbalances[-3:]

def delete_down_imbalance(down_imbalances, df:pd.DataFrame)->list:
    for i in range(0, len(df)-10):
        for imbalance in down_imbalances:
            candle = df.iloc[i]
            if candle['date']>imbalance[2] and (candle['close'] > imbalance[0] or candle['high'] > imbalance[0]):
                down_imbalances.remove(imbalance)
    
    return down_imbalances[-3:]

def add_moving_average(period:int,df:pd.DataFrame)->pd.DataFrame:
    df['SMA'] = df['close'].rolling(period).mean()
    df['SMAslope'] = df['SMA'].pct_change()*100

    df['up_reversal'] = ((df['SMAslope'].shift(1) < 0) & (df['SMAslope'] > 0))
    df['down_reversal'] = ((df['SMAslope'].shift(1) > 0) & (df['SMAslope'] < 0)) 
    # df['charthighest'] = df['high'].rolling(6).max()[df['down_reversal'] == True]
    # df['chartlowest'] = df['low'].rolling(6).min()[df['up_reversal'] == True]
    # up_imbalances = get_list_of_up_imbalances(df)
    # down_imbalances = get_list_of_down_imbalances(df)

    # df['up_reversal'] = False
    # df['down_reversal'] = False
    # for i in range(5, len(df)-3):
    #     for imbalance in up_imbalances:
    #         if (df.iloc[i-1]['date'] < imbalance[2] < df.iloc[i+3]['date']):
    #             if (df.iloc[i-1]['SMAslope'] < 0) & (df.iloc[i]['SMAslope'] > 0):
    #                 if not (df.iloc[i-1]['up_reversal'] or df.iloc[i-2]['up_reversal'] or df.iloc[i-3]['up_reversal'] or df.iloc[i-4]['up_reversal']):
    #                     df.loc[i, 'up_reversal'] = True
    #     for imbalance in down_imbalances:
    #         if (df.iloc[i-1]['date'] < imbalance[2] < df.iloc[i+3]['date']):
    #             if ((df.iloc[i-1]['SMAslope'] > 0) & (df.iloc[i]['SMAslope'] < 0)):
    #                 if not (df.iloc[i-1]['down_reversal'] or df.iloc[i-2]['down_reversal'] or df.iloc[i-3]['down_reversal'] or df.iloc[i-4]['down_reversal']):
    #                     df.loc[i, 'down_reversal'] = True


    # df['up_reversal'] = ((df['SMAslope'].shift(1) < 0) & (df['SMAslope'] > 0))
    # df['down_reversal'] = ((df['SMAslope'].shift(1) > 0) & (df['SMAslope'] < 0))

    return df

def add_rsx_HF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSXslope'].shift(1).rolling(5).sum()<-6)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSXslope'].shift(1).rolling(5).sum()>6)

    up_reversals = []
    down_reversals = []
    df['hidden_up_divergence'] = False
    df['hidden_down_divergence'] = False

    for i in range(0, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if (up_reversals and down_reversals):
            for up_reversal in up_reversals[-5:]:
                if (up_reversal[0] < df.iloc[i]['low']) and (up_reversal[1] > df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'hidden_up_divergence'] = True
                
            for down_reversal in down_reversals[-5:]:
                if (down_reversal[0] > df.iloc[i]['high']) and (down_reversal[1] < df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'hidden_down_divergence'] = True

    return df

def add_rsx_LF(period:int,df:pd.DataFrame)->pd.DataFrame:
    df["RSX"] = ta.rsx(round((df['close']+df['high']+df['low'])/3,2), period)
    df['RSXslope'] = df['RSX'].pct_change()*100
    df['RSX_up_reversal'] = ((df['RSXslope'].shift(1) < 0) & (df['RSXslope'] > 0)) & (df['RSXslope'].shift(1).rolling(5).sum()<-6)
    df['RSX_down_reversal'] = ((df['RSXslope'].shift(1) > 0) & (df['RSXslope'] < 0)) & (df['RSXslope'].shift(1).rolling(5).sum()>6)

    up_reversals = []
    down_reversals = []
    df['classic_up_divergence'] = False
    df['classic_down_divergence'] = False

    for i in range(len(df) - 200, len(df)):
        if df.iloc[i]['RSX_up_reversal']:
            up_reversals.append(min((df.iloc[i]['low'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['low'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['low'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['low'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['low'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['low'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))

        if df.iloc[i]['RSX_down_reversal']:
            down_reversals.append(max((df.iloc[i]['high'],df.iloc[i-1]['RSX'],df.iloc[i]['date']),
                (df.iloc[i-1]['high'],df.iloc[i-1]['RSX'],df.iloc[i-1]['date']),
                (df.iloc[i-2]['high'],df.iloc[i-1]['RSX'],df.iloc[i-2]['date']),
                (df.iloc[i-3]['high'],df.iloc[i-1]['RSX'],df.iloc[i-3]['date']),
                (df.iloc[i-4]['high'],df.iloc[i-1]['RSX'],df.iloc[i-4]['date']),
                (df.iloc[i-5]['high'],df.iloc[i-1]['RSX'],df.iloc[i-5]['date'])))
                
        if (up_reversals and down_reversals):
            for up_reversal in up_reversals[-1:]:    
                if (up_reversal[0] > df.iloc[i]['low']) and (up_reversal[1] < df.iloc[i]['RSX']) and up_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_up_divergence'] = True

            for down_reversal in down_reversals[-1:]:       
                if (down_reversal[0] < df.iloc[i]['high']) and (down_reversal[1] > df.iloc[i]['RSX']) and down_reversal[2] < df.iloc[i]['date']:
                    df.loc[i, 'classic_down_divergence'] = True
                
    return df

In [10]:
# up_imbalances = get_list_of_up_imbalances(df_higher_tf)
# down_imbalances = get_list_of_down_imbalances(df_higher_tf)
# up_imbalances = delete_up_imbalance(up_imbalances,df_higher_tf)
# down_imbalances = delete_down_imbalance(down_imbalances,df_higher_tf)
df_higher_tf = get_price_data_binance('BTCUSDT',200,interval=Client.KLINE_INTERVAL_1DAY)
df_lower_tf = get_price_data_binance('BTCUSDT',200,interval=Client.KLINE_INTERVAL_6HOUR)
df_lower_tf = add_rsx_LF(21,df_lower_tf)
df_lower_tf = add_moving_average(3,df_lower_tf)
df_higher_tf = add_rsx_HF(21,df_higher_tf)

fig = go.Figure(data=[go.Candlestick(x=df_higher_tf['date'],
                open=df_higher_tf['open'],
                high=df_higher_tf['high'],
                low=df_higher_tf['low'],
                close=df_higher_tf['close'])])

fig1 = go.Figure(data=[go.Candlestick(x=df_lower_tf['date'],
                open=df_lower_tf['open'],
                high=df_lower_tf['high'],
                low=df_lower_tf['low'],
                close=df_lower_tf['close'])])

for i in range(1, 10):
    if df_lower_tf.iloc[-i]['up_reversal']:
        if df_higher_tf.iloc[-i]['hidden_up_divergence']:
            if df_lower_tf.iloc[-i]['classic_up_divergence']:
                fig1.add_vline(df_lower_tf.iloc[-i]['date'],line_color="green")

    if df_lower_tf.iloc[-i]['down_reversal']:
        if df_higher_tf.iloc[-i]['hidden_down_divergence']:
            if df_lower_tf.iloc[-i]['classic_down_divergence']:
                fig1.add_vline(df_lower_tf.iloc[-i]['date'],line_color="red")

rsx = go.Figure(go.Scatter(
        x=df_higher_tf['date'],
        y=df_higher_tf['RSX']
    ))

fig.update_layout(showlegend=False)
fig.show()
fig1.update_layout(showlegend=False)
fig1.show()
rsx.update_layout(showlegend=False)
rsx.show()

In [9]:
df_lower_tf

,date,open,high,low,close,volume,RSX,RSXslope,RSX_up_reversal,RSX_down_reversal,classic_up_divergence,classic_down_divergence,SMA,SMAslope,up_reversal,down_reversal
0,2022-12-01 18:00:00,16967.67,16987.29,16855.01,16977.37,39344.92446000,NaN,NaN,False,False,False,False,NaN,NaN,False,False
1,2022-12-02 00:00:00,16978.00,17048.99,16857.89,16934.84,50620.33530000,NaN,NaN,False,False,False,False,NaN,NaN,False,False
2,2022-12-02 06:00:00,16935.46,17065.00,16906.99,16996.92,45317.17736000,NaN,NaN,False,False,False,False,16969.710000,NaN,False,False
3,2022-12-02 12:00:00,16996.92,17105.73,16787.85,16936.33,69738.93158000,NaN,NaN,False,False,False,False,16956.030000,-0.080614,False,False
4,2022-12-02 18:00:00,16935.68,17099.00,16930.03,17092.74,36695.76196000,NaN,NaN,False,False,False,False,17008.663333,0.310411,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2023-01-19 12:00:00,20744.17,20960.23,20660.80,20841.16,84846.90404000,64.017050,-5.114435,False,False,False,True,20799.900000,0.263015,True,False
196,2023-01-19 18:00:00,20841.16,21192.00,20831.08,21071.59,64900.36473000,61.103137,-4.551776,False,False,False,True,20885.490000,0.411492,False,False
197,2023-01-20 00:00:00,21071.59,21219.60,20953.23,20982.27,57167.91217000,58.716820,-3.905391,False,False,False,True,20965.006667,0.380727,False,False
198,2023-01-20 06:00:00,20983.09,20999.80,20861.28,20963.51,51741.71150000,56.460854,-3.842112,False,False,False,True,21005.790000,0.194531,False,False
